# Demo

let's analyze few homes 


### step 1: who might be eligible 

Santa Clara County 车库改造 + detached adu

Alameda County - 车库改造 + detached adu

la county - 车库改造最多

San Francisco County - 车库改造居多


### find price per sqft high and total price not high
# lot size / home size; 
total price / price per sqft





In [5]:
# Read JSON data from county files
import json
import pandas as pd
import os
pd.set_option('display.max_colwidth', 300)

In [6]:
# Define file paths
data_dir = "../data/raw/"
county_files = {
    "Santa Clara": "20250822_114114_santa_clara_county_ca_santa_clara_county.json",
    "Alameda": "20250822_114052_alameda_county_ca_alameda_county.json", 
    "Los Angeles": "20250822_113940_los_angeles_county_ca_los_angeles_county.json"
}

# Load data from each county
county_data = {}
for county_name, filename in county_files.items():
    filepath = os.path.join(data_dir, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        county_data[county_name] = data
        print(f"Loaded {county_name} County: {len(data['listings'])} listings")

# Convert listings to DataFrames for easier analysis
county_dfs = {}
for county_name, data in county_data.items():
    # Extract relevant fields from each listing
    listings = []
    for listing in data['listings']:
        listing_data = {
            'address': listing.get('location', {}).get('address', {}).get('line', ''),
            'city': listing.get('location', {}).get('address', {}).get('city', ''),
            'zip': listing.get('location', {}).get('address', {}).get('postal_code', ''),
            'price': listing.get('list_price', 0),
            'price_per_sqft': listing.get('price_per_sqft', 0),
            'sqft': listing.get('description', {}).get('sqft', 0),
            'lot_sqft': listing.get('description', {}).get('lot_sqft', 0),
            'beds': listing.get('description', {}).get('beds', 0),
            'baths': listing.get('description', {}).get('baths', 0),
            #add text
            'description': listing.get('description', {}).get('text', ''),
            'year_built': listing.get('description', {}).get('year_built', 0),
            'garage': listing.get('description', {}).get('garage', 0),
            'property_type': listing.get('description', {}).get('type', ''),
            'listing_id': listing.get('listing_id', ''),
            'url': listing.get('href', '')
        }
        listings.append(listing_data)
    
    county_dfs[county_name] = pd.DataFrame(listings)
    print(f"{county_name} DataFrame created with shape: {county_dfs[county_name].shape}")

# Display basic stats for each county
for county_name, df in county_dfs.items():
    print(f"\n{county_name} County Statistics:")
    print(f"  Average price: ${df['price'].mean():,.0f}")
    print(f"  Average price/sqft: ${df['price_per_sqft'].mean():,.0f}")
    print(f"  Average sqft: {df['sqft'].mean():,.0f}")
    print(f"  Average lot sqft: {df['lot_sqft'].mean():,.0f}")

#conver to pandas dataframe
county_dfs = pd.concat(county_dfs.values())

#display width change to 1000
pd.set_option('display.max_colwidth', 300)

#display
county_dfs.head(10)


Loaded Santa Clara County: 1366 listings
Loaded Alameda County: 1841 listings
Loaded Los Angeles County: 4513 listings
Santa Clara DataFrame created with shape: (1366, 15)
Alameda DataFrame created with shape: (1841, 15)
Los Angeles DataFrame created with shape: (4513, 15)

Santa Clara County Statistics:
  Average price: $2,192,131
  Average price/sqft: $0
  Average sqft: 2,104
  Average lot sqft: 28,001

Alameda County Statistics:
  Average price: $1,557,264
  Average price/sqft: $0
  Average sqft: 2,202
  Average lot sqft: 229,482

Los Angeles County Statistics:
  Average price: $1,346,727
  Average price/sqft: $0
  Average sqft: 2,073
  Average lot sqft: 465,014


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,url
0,5939 Pala Mesa Dr,San Jose,95123,1899888,0,2451.0,4715.0,4,3.0,"Welcome to 5939 Pala Mesa Drive in the desirable Palmia community. This 4-bedroom, 2.5-bath home offers approximately 2, 451 square feet of well-designed living space. The open layout includes a welcoming living room, formal dining area, and a cozy family room with a gas fireplace. Brazil-engine...",2000.0,0,single_family,2985606552,https://www.realtor.com/realestateandhomes-detail/5939-Pala-Mesa-Dr_San-Jose_CA_95123_M28956-69893
1,119 Belcrest Dr,Los Gatos,95032,2699000,0,1748.0,10376.0,4,2.0,"Beautifully renovated single-story gem located in the sought-after Belwood of Los Gatos community. This exquisite home is situated on a nearly quarter-acre lot, providing privacy and space for fun & relaxation. Step inside to discover a harmonious blend of elegance and practicality. The kitchen ...",1965.0,0,single_family,2985604175,https://www.realtor.com/realestateandhomes-detail/119-Belcrest-Dr_Los-Gatos_CA_95032_M29058-05098
2,2055 Rucker Ave,Gilroy,95020,1649777,0,1172.0,306662.0,3,2.0,"Brand-new home on 7 flat acres + permits for a 4, 500 sq ft 2nd home! 2 addresses/meters, seasonal stream, prime New Ave frontage. Live in or rent the new home while you build endless possibilities! Brand-New Home + Permits for a Second 4, 500 Sq. Ft. Estate on 7 Usable Acres! Move right in to y...",2025.0,0,single_family,2985604058,https://www.realtor.com/realestateandhomes-detail/2055-Rucker-Ave_Gilroy_CA_95020_M27172-67002
3,5850 Chesbro Ave,San Jose,95123,1499900,0,1777.0,6000.0,3,3.0,"Welcome to 5850 Chesbro Ave, a beautifully remodeled home in the heart of Blossom Valley. This open-concept residence blends modern design with comfortable living, featuring a bright and spacious layout perfect for todays lifestyle. The chefs kitchen showcases sleek cabinetry, quartz countertops...",1964.0,0,single_family,2985597086,https://www.realtor.com/realestateandhomes-detail/5850-Chesbro-Ave_San-Jose_CA_95123_M18665-35552
4,1641 Rue Avati,San Jose,95131,1349888,0,1205.0,6247.0,3,2.0,"Welcome to 1641 Rue Avati, San Jose, CA 95131. This beautifully upgraded North San Jose home offers the perfect blend of modern comfort and unbeatable convenience. Featuring 3 bedrooms, 2 bathrooms, and 1, 205 sq ft of living space, this move-in-ready gem is packed with brand-new upgrades throug...",1973.0,0,single_family,2985596768,https://www.realtor.com/realestateandhomes-detail/1641-Rue-Avati_San-Jose_CA_95131_M14901-29936
5,10163 Bret Ave,Cupertino,95014,4700000,0,3323.0,9375.0,5,4.0,"Welcome to this spacious 5-bedroom, 3-bathroom home located in the vibrant city of Cupertino. Spanning 3, 323 square feet, this property offers ample space for comfortable living. The well-appointed kitchen is designed for functionality, perfect for the home chef. Enjoy the warmth of the firepla...",1994.0,0,single_family,2985596657,https://www.realtor.com/realestateandhomes-detail/10163-Bret-Ave_Cupertino_CA_95014_M26842-44549
6,1651 Cabrillo Ave,Santa Clara,95050,990000,0,985.0,5000.0,3,1.0,"Welcome to this conveniently located single-family home in Santa Clara. The inviting living room features large windows, while the gourmet kitchen offers quartz countertops, a stylish backsplash, appliances, and ample cabinetry. A dining area with a beautiful ceiling light completes the space. T...",1954.0,0,single_family,2985596505,https://www.realtor.com/realestateandhomes-detail/1651-Cabrillo-Ave_Santa-Clara_CA_95050_M12563-74069
7,5991 Sorrel Ave,San Jose,95123,1600000,0,1952.0,6120.0,5,2.0,"Welcome to 5991 Sorrel Avenue, a rare single-story, five-bedroom residence offering refined comfort and modern efficiency. With dual living spaces, tasteful updates, a beautifully landscaped backyard, and a Tesla solar system with whole-home battery backup, this home is designed for both style a...",1965.0,0,single_family,2985

In [7]:
#display
county_dfs.head(10)

,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,url
0,5939 Pala Mesa Dr,San Jose,95123,1899888,0,2451.0,4715.0,4,3.0,"Welcome to 5939 Pala Mesa Drive in the desirable Palmia community. This 4-bedroom, 2.5-bath home offers approximately 2, 451 square feet of well-designed living space. The open layout includes a welcoming living room, formal dining area, and a cozy family room with a gas fireplace. Brazil-engine...",2000.0,0,single_family,2985606552,https://www.realtor.com/realestateandhomes-detail/5939-Pala-Mesa-Dr_San-Jose_CA_95123_M28956-69893
1,119 Belcrest Dr,Los Gatos,95032,2699000,0,1748.0,10376.0,4,2.0,"Beautifully renovated single-story gem located in the sought-after Belwood of Los Gatos community. This exquisite home is situated on a nearly quarter-acre lot, providing privacy and space for fun & relaxation. Step inside to discover a harmonious blend of elegance and practicality. The kitchen ...",1965.0,0,single_family,2985604175,https://www.realtor.com/realestateandhomes-detail/119-Belcrest-Dr_Los-Gatos_CA_95032_M29058-05098
2,2055 Rucker Ave,Gilroy,95020,1649777,0,1172.0,306662.0,3,2.0,"Brand-new home on 7 flat acres + permits for a 4, 500 sq ft 2nd home! 2 addresses/meters, seasonal stream, prime New Ave frontage. Live in or rent the new home while you build endless possibilities! Brand-New Home + Permits for a Second 4, 500 Sq. Ft. Estate on 7 Usable Acres! Move right in to y...",2025.0,0,single_family,2985604058,https://www.realtor.com/realestateandhomes-detail/2055-Rucker-Ave_Gilroy_CA_95020_M27172-67002
3,5850 Chesbro Ave,San Jose,95123,1499900,0,1777.0,6000.0,3,3.0,"Welcome to 5850 Chesbro Ave, a beautifully remodeled home in the heart of Blossom Valley. This open-concept residence blends modern design with comfortable living, featuring a bright and spacious layout perfect for todays lifestyle. The chefs kitchen showcases sleek cabinetry, quartz countertops...",1964.0,0,single_family,2985597086,https://www.realtor.com/realestateandhomes-detail/5850-Chesbro-Ave_San-Jose_CA_95123_M18665-35552
4,1641 Rue Avati,San Jose,95131,1349888,0,1205.0,6247.0,3,2.0,"Welcome to 1641 Rue Avati, San Jose, CA 95131. This beautifully upgraded North San Jose home offers the perfect blend of modern comfort and unbeatable convenience. Featuring 3 bedrooms, 2 bathrooms, and 1, 205 sq ft of living space, this move-in-ready gem is packed with brand-new upgrades throug...",1973.0,0,single_family,2985596768,https://www.realtor.com/realestateandhomes-detail/1641-Rue-Avati_San-Jose_CA_95131_M14901-29936
5,10163 Bret Ave,Cupertino,95014,4700000,0,3323.0,9375.0,5,4.0,"Welcome to this spacious 5-bedroom, 3-bathroom home located in the vibrant city of Cupertino. Spanning 3, 323 square feet, this property offers ample space for comfortable living. The well-appointed kitchen is designed for functionality, perfect for the home chef. Enjoy the warmth of the firepla...",1994.0,0,single_family,2985596657,https://www.realtor.com/realestateandhomes-detail/10163-Bret-Ave_Cupertino_CA_95014_M26842-44549
6,1651 Cabrillo Ave,Santa Clara,95050,990000,0,985.0,5000.0,3,1.0,"Welcome to this conveniently located single-family home in Santa Clara. The inviting living room features large windows, while the gourmet kitchen offers quartz countertops, a stylish backsplash, appliances, and ample cabinetry. A dining area with a beautiful ceiling light completes the space. T...",1954.0,0,single_family,2985596505,https://www.realtor.com/realestateandhomes-detail/1651-Cabrillo-Ave_Santa-Clara_CA_95050_M12563-74069
7,5991 Sorrel Ave,San Jose,95123,1600000,0,1952.0,6120.0,5,2.0,"Welcome to 5991 Sorrel Avenue, a rare single-story, five-bedroom residence offering refined comfort and modern efficiency. With dual living spaces, tasteful updates, a beautifully landscaped backyard, and a Tesla solar system with whole-home battery backup, this home is designed for both style a...",1965.0,0,single_family,2985

### Step 2: Check Zone and city code
